In [568]:
%run "./metrics/metrics"

In [308]:
import numpy as np

class Bootstrap():
    def __init__(self, X, y, n_samples, len_sample):
        self.n_samples = n_samples
        self.len_sample = len_sample
        self.X = np.array(X)
        self.y = np.array(y)
        self.sample_X = np.zeros((n_samples, len_sample, self.X.shape[1]))
        self.sample_y = np.zeros((n_samples, len_sample, self.y.shape[1]))
        for i in range(n_samples):
            idx = np.random.choice(np.arange(len(X)), len_sample, replace=True)
            self.sample_X[i] = self.X[idx]
            self.sample_y[i] = self.y[idx]
        #print('a', *self.sample_X)
        #print('b', *self.sample_y)
    
    def values(self):
        return self.sample_X, self.sample_y
        


In [309]:
X = [[1, 2], [3, 4], [5, 6]]
b = Bootstrap(X, X, 5, 3)


In [342]:
class Model():
    def __init__(self):
        pass
    def fit(self, X, y):
        pass
    def predict(self, X):
        pass
    def predict_params(self, X):
        pass
        

In [506]:
%run "./metrics/metrics"

In [549]:
class FunctionEvaluator():
    def __init__(self, func=None):
        if func == None:
            self.function = lambda a, b: 0
        else:
            self.function = func

In [565]:
class Metric(FunctionEvaluator):
    def evaluate(self, model, X, y):
        z = self.function(y, model.predict(X))
        return z
    

In [551]:
class Characteristic(FunctionEvaluator):
    def evaluate(self, model):
        pass
    

In [552]:
class Comparison(FunctionEvaluator):
    def evaluate(self, full_model, model, full_X, y, mask):
        pass

In [553]:
class RSS(Metric):
    def __init__(self):
        super(RSS, self).__init__(residual_square_sum)

In [554]:
class Adjusted_Determination_Coefficient(Metric):
    def __init__(self):
        super(Adjusted_Determination_Coefficient, self).__init__(determination_coefficient)
        
class Determination_Coefficient(Metric):
    def __init__(self):
        super(Determination_Coefficient, self).__init__(determination_coefficient)
    def evaluate(self, model, X, y):
        z = self.function(y, model.predict(X), False)
        #print("Z", z)
        return z

In [555]:
class VIF(Metric):
    def __init__(self):
        super(VIF, self).__init__(variance_inflation_factor)

In [567]:
class BIC(Metric):
    def __init__(self):
        super(BIC, self).__init__(bayesian_information_criterion)
    def evaluate(self, model, X, y):
        num_features = X.shape[1]
        z = self.function(y, model.predict(X), num_features)
        #print("Z", z)
        return z

In [574]:
class Cp(Comparison):
    def __init__(self):
        super(Cp, self).__init__(mallows_Cp)
    def evaluate(self, full_model, model, full_X, y, mask):
        y_full = full_model.predict(full_X)
        y_p = model.predict((full_X.T[mask]).T)
        p = mask.sum()
        return self.function(y, y_full, y_p, p)
        
        

In [592]:
class Condition_Number(Characteristic):
    def __init__(self):
        super(Condition_Number, self).__init__(condition_number_xtx)
    def evaluate(self, model):
        X = model.coef_
        return self.function(X)

In [593]:
c = Condition_Number()
c.evaluate(model)

25.888917585823872

In [557]:
class EvaluateInfo():
    def __init__(self, model, metrics = [], characteristics = [], comparisons = []):
        self.model = model
        self.characteristics = characteristics # characteristic of the fitted model itself
        self.metrics = metrics # compares y and y_pred
        self.comparisons = comparisons #compares y, y_pred, y_pred_with_reduced_features
        
    def fit(self, X_train, y_train, X_test, y_test, mask = None, n_samples=20, len_sample=None):
        if mask == None:
            mask = np.ones(X.shape[1], dtype=np.bool)
        self.mask = mask
        if len_sample is None:
            len_sample = X.shape[1]
        self.len_sample = len_sample
        self.n_samples = n_samples

        self.X_train = X_train
        self.y_train = y_train

        self.X_test = X_test
        self.y_test = y_test

In [581]:
from copy import deepcopy
class EvaluateStaticInfo(EvaluateInfo):
    def __init__(self, model, metrics = [], comparisons = []):
        super(EvaluateStaticInfo, self).__init__(model, metrics, [], comparisons)
    def __eval(self):
        self.sample_X, self.sample_y = self.b.values()
        result_metrics = np.zeros((len(self.metrics), self.n_samples))
        result_comparisons = np.zeros((len(self.comparisons), self.n_samples))
        self.full = deepcopy(model.fit(self.X_train, self.y_train))
        
        model.fit((self.X_train.T[self.mask]).T, self.y_train)
        
        for it in range(self.n_samples):
            for (i, metric) in enumerate(self.metrics):
                result_metrics[i][it] = metric.evaluate(model, (self.sample_X[it].T[self.mask]).T, self.sample_y[it])
            for(i, comp) in enumerate(self.comparisons):
                result_comparisons[i][it] = comp.evaluate(self.full, model, self.sample_X[it], self.sample_y[it], self.mask)
                
        return result_metrics

    
    def fit(self, X_train, y_train, X_test, y_test, mask = None, n_samples=20, len_sample=None):
        super(EvaluateStaticInfo, self).fit(X_train, y_train, X_test, y_test, mask, n_samples, len_sample)
        self.b = Bootstrap(self.X_test, self.y_test, self.n_samples, self.len_sample)
        return self.__eval()
        

In [582]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [583]:
import scipy.stats as sps

X = sps.norm.rvs(size=(10, 5))
X2 = sps.norm.rvs(size=(10, 5))
z = sps.uniform.rvs(size=(5, 3))
y = X @ z + sps.norm.rvs(scale=0.1, size=(10, 3))
y2 = X2 @ z + sps.norm.rvs(scale=0.1, size=(10, 3))


In [584]:
metric1 = RSS()
metric2 = Adjusted_Determination_Coefficient()
metric3 = Determination_Coefficient()
metric4 = VIF()
metric5 = BIC()
comp1 = Cp()

In [585]:
info = EvaluateStaticInfo(model=model, metrics = [metric1, metric2, metric3, metric4, metric5], comparisons=[comp1])

In [586]:
info.fit(X, y, X2, y2)

array([[   0.89623303,    0.97802173,    0.98327159,    1.0422683 ,
           0.86208165,    0.58751302,    1.0855854 ,    0.88037748,
           0.50445093,    1.18274876,    0.73847399,    0.79795139,
           1.30277444,    0.81667243,    1.21353461,    1.22919977,
           0.87725362,    0.65079803,    0.42040033,    1.09234467],
       [   0.98338471,    0.98916786,    0.98592616,    0.9725401 ,
           0.97289551,    0.96686773,    0.96614138,    0.93739002,
           0.98200029,    0.96692171,    0.97383809,    0.98155521,
           0.93427608,    0.99265358,    0.9803056 ,    0.96463124,
           0.97077176,    0.97763501,    0.98555407,    0.98062596],
       [   0.96676942,    0.97833572,    0.97185232,    0.9450802 ,
           0.94579101,    0.93373546,    0.93228275,    0.87478005,
           0.96400058,    0.93384342,    0.94767618,    0.96311043,
           0.86855215,    0.98530717,    0.96061121,    0.92926248,
           0.94154353,    0.95527003,    0.971